# Build the filters and data for the LAE/LBG templates project.


In [5]:
import os
import numpy as np
import fitsio
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt

In [6]:
datadir = '/Users/ioannis/research/projects/fastspecfit-projects/laelbg-templates'

In [7]:
def gather_cosmos2020_filters():
    """
    https://cosmos.astro.caltech.edu/page/filterset
    https://eazy-py.readthedocs.io/en/latest/eazy/filters.html
    https://github.com/cosmic-dawn/cosmos2020-readcat/blob/main/COSMOS2020_readcat.ipynb    
    
    GALEX_FUV - 120 CAPAK/galex1500.res FUV lambda_c= 1.5364e+03 AB-Vega= 2.128 w95=372.4  
    GALEX_NUV - 121 CAPAK/galex2500.res NUV lambda_c= 2.2992e+03 AB-Vega= 1.665 w95=925.3  
    CFHT_u - 348 CFHT_Megaprime.u_sdss.dat-cfh9302 lambda_c= 3.54017e+03  
    CFHT_ustar - 88 megaprime/cfht_mega_u_cfh9301.dat CFHT-LS+atm lambda_c= 3.8280e+03 AB-Vega= 0.325 w95=771.0  
    HSC_g - 314 g_HSC.txt http://cosmos.astro.caltech.edu/page/filterset lambda_c=4798.2 AB-Vega=-0.089  
    HSC_r - 315 r_HSC.txt http://cosmos.astro.caltech.edu/page/filterset lambda_c=6218.4 AB-Vega=0.223  
    HSC_i - 316 i_HSC.txt http://cosmos.astro.caltech.edu/page/filterset lambda_c=7727.0 AB-Vega=0.468  
    HSC_z - 317 z_HSC.txt http://cosmos.astro.caltech.edu/page/filterset lambda_c=8908.2 AB-Vega=0.522  
    HSC_y - 318 y_HSC.txt http://cosmos.astro.caltech.edu/page/filterset lambda_c=9775.1 AB-Vega=0.740  
    UVISTA_Y - 256 VISTA/Y_system+atmos.dat at80K_forETC+trans_10_10 lambda_c= 1.0217e+04 AB-Vega= 0.596 w95=1025.7  
    UVISTA_J - 257 VISTA/J_system+atmos.dat lambda_c= 1.2527e+04 AB-Vega= 0.911 w95=1703.1  
    UVISTA_H - 258 VISTA/H_system+atmos.dat lambda_c= 1.6433e+04 AB-Vega= 1.355 w95=2843.9  
    UVISTA_Ks - 259 VISTA/Ks_system+atmos.dat lambda_c= 2.1503e+04 AB-Vega= 1.819 w95=3108.7  
    SC_IB427 - 181 Subaru_MB/IA427.dat lambda_c= 4.2600e+03 AB-Vega=-0.161 w95=223.1  
    SC_IB464 - 183 Subaru_MB/IA464.dat lambda_c= 4.6333e+03 AB-Vega=-0.167 w95=237.8  
    SC_IA484 - 184 Subaru_MB/IA484.dat lambda_c= 4.8473e+03 AB-Vega=-0.037 w95=249.5  
    SC_IB505 - 185 Subaru_MB/IA505.dat lambda_c= 5.0608e+03 AB-Vega=-0.077 w95=259.0  
    SC_IA527 - 186 Subaru_MB/IA527.dat lambda_c= 5.2593e+03 AB-Vega=-0.035 w95=281.7  
    SC_IB574 - 188 Subaru_MB/IA574.dat lambda_c= 5.7629e+03 AB-Vega= 0.054 w95=303.3  
    SC_IA624 - 190 Subaru_MB/IA624.dat lambda_c= 6.2308e+03 AB-Vega= 0.142 w95=336.6  
    SC_IA679 - 192 Subaru_MB/IA679.dat lambda_c= 6.7816e+03 AB-Vega= 0.245 w95=371.9  
    SC_IB709 - 193 Subaru_MB/IA709.dat lambda_c= 7.0735e+03 AB-Vega= 0.287 w95=358.3  
    SC_IA738 - 194 Subaru_MB/IA738.dat lambda_c= 7.3595e+03 AB-Vega= 0.334 w95=355.0  
    SC_IA767 - 195 Subaru_MB/IA768.dat lambda_c= 7.6804e+03 AB-Vega= 0.387 w95=388.5  
    SC_IB827 - 197 Subaru_MB/IA827.dat lambda_c= 8.2468e+03 AB-Vega= 0.475 w95=367.0  
    SC_NB711 - 322 COSMOS/SUBARU_filter_NB711.txt lambda_c= 7.1202e+03 AB-Vega= xxxx  
    SC_NB816 - 84 COSMOS/SUBARU_filter_NB816.txt lambda_c= 8.1509e+03 AB-Vega= 0.461 w95=162.0  
    UVISTA_NB118 - 321 VISTA-NB118 atm lambda_c= 1.1909e+04 AB-Vega= xxxx  
    SC_B - 78 COSMOS/SUBARU_filter_B.txt lambda_c= 4.4480e+03 AB-Vega=-0.112 w95=1035.4  
    SC_gp - 80 COSMOS/SUBARU_filter_g.txt lambda_c= 4.7609e+03 AB-Vega=-0.101 w95=1317.6  
    SC_V - 79 COSMOS/SUBARU_filter_V.txt lambda_c= 5.4702e+03 AB-Vega=-0.000 w95=993.1  
    SC_rp - 116 CAPAK_v2/r_subaru.res cosmos-r lambda_c= 6.2755e+03 AB-Vega= 0.154 w95=1379.4  
    SC_ip - 117 CAPAK_v2/i_subaru.res cosmos-i lambda_c= 7.6712e+03 AB-Vega= 0.380 w95=1488.9  
    SC_zp - 118 CAPAK_v2/z_subaru.res cosmos-z lambda_c= 9.0282e+03 AB-Vega= 0.514 w95=1410.5  
    SC_zpp - 284 Subaru/suprime_FDCCD_z.res lambda_c= 9.0963e+03 AB-Vega=0.516 w95=1424.0  
    IRAC_CH1 - 18 IRAC/irac_tr1_2004-08-09.dat 3.6micron lambda_c= 3.5569e+04 AB-Vega= 2.781 w95=7139.2  
    IRAC_CH2 - 19 IRAC/irac_tr2_2004-08-09.dat 4.5micron lambda_c= 4.5020e+04 AB-Vega= 3.254 w95=9705.5  
    IRAC_CH3 - 20 IRAC/irac_tr3_2004-08-09.dat 5.8micron lambda_c= 5.7450e+04 AB-Vega= 3.747 w95=13590.7  
    IRAC_CH4 - 21 IRAC/irac_tr4_2004-08-09.dat 8.0micron lambda_c= 7.9158e+04 AB-Vega= 4.387 w95=27892.8      
    
    """
    # Filter names, mean wavelength, and other info (see Table 1 in Weaver+22)
    filt_name = ['GALEX_FUV', 'GALEX_NUV','CFHT_u','CFHT_ustar','HSC_g', 'HSC_r', 'HSC_i', 
                 'HSC_z', 'HSC_y', 'UVISTA_Y', 'UVISTA_J', 'UVISTA_H', 'UVISTA_Ks', 
                 'SC_IB427', 'SC_IB464', 'SC_IA484', 'SC_IB505', 'SC_IA527', 'SC_IB574', 
                 'SC_IA624', 'SC_IA679', 'SC_IB709', 'SC_IA738', 'SC_IA767', 'SC_IB827', 
                 'SC_NB711', 'SC_NB816', 'UVISTA_NB118', 'SC_B', 'SC_gp', 'SC_V', 'SC_rp', 
                 'SC_ip','SC_zp', 'SC_zpp', 'IRAC_CH1', 'IRAC_CH2', 'IRAC_CH3','IRAC_CH4']  
    
    eazy_filtnum = [120, 121, 348, 88, 314, 315, 316, 317, 318, 256, 257, 258, 259, 181, 
                    183, 184, 185, 186, 188, 190, 192, 193, 194, 195, 197, 322, 84, 321, 
                    78, 80, 79, 116, 117, 118, 284, 18, 19, 20, 21]
    
    filt_lambda = [0.1526, 0.2307, 0.3709, 0.3858, 0.4847, 0.6219, 0.7699, 0.8894, 0.9761, 
                   1.0216, 1.2525, 1.6466,2.1557, 0.4266, 0.4635, 0.4851, 0.5064, 0.5261, 
                   0.5766, 0.6232, 0.6780, 0.7073, 0.7361, 0.7694, 0.8243, 0.7121, 0.8150, 
                   1.1909, 0.4488, 0.4804, 0.5487, 0.6305, 0.7693, 0.8978, 0.9063, 3.5686, 
                   4.5067,5.7788,7.9958]
    
    filt_fwhm = [0.0224, 0.07909, 0.05181, 0.05976, 0.1383, 0.1547, 0.1471, 0.0766, 0.0786, 
                 0.0923, 0.1718, 0.2905, 0.3074, 0.02073, 0.02182, 0.02292, 0.0231, 0.02429, 
                 0.02729, 0.03004, 0.03363, 0.03163, 0.03235, 0.03648, 0.0343, 0.0072, 0.01198, 
                 0.01122, 0.0892, 0.1265, 0.0954, 0.1376, 0.1497, 0.0847, 0.1335, 0.7443, 1.0119, 
                 1.4082,2.8796] 
    
    AlambdaDivEBV = [8.31,8.742,4.807,4.674,3.69,2.715,2.0, 1.515, 1.298, 1.213, 0.874, 0.565, 
                     0.365,4.261,3.844,3.622,3.425,3.265,2.938,2.694,2.431,2.29,2.151, 1.997, 
                     1.748,2.268, 1.787, 0.946,4.041,3.738,3.128,2.673,2.003, 1.436, 1.466, 
                     0.163, 0.112, 0.075, 0.045]
    
    # photometric offsets (not available for all filters, see Table 3 in W+21)
    zpoff1 = [0.000, -0.352, -0.077, -0.023, 0.073, 0.101, 0.038, 0.036, 0.086, 0.054, # Farmer+LePhare
              0.017, -0.045, 0.000, -0.104, -0.044, -0.021, -0.018, -0.045, -0.084, 0.005, 
              0.166, -0.023, -0.034, -0.032, -0.069, -0.010, -0.064, 0.000, 0.000, 0.000, 
              0.000, 0.000, 0.000, 0.000, 0.000, -0.212, -0.219, 0.000, 0.000]  
    
    zpoff2 = [0.000, -0.029, -0.006, 0.053, 0.128, 0.127, 0.094, 0.084, 0.100, # Classic+LePhare
              0.049, 0.025, -0.044, 0.000, -0.013, -0.008, 0.022, 0.025, 0.033, 
              -0.032, 0.031, 0.208, -0.009, 0.003, -0.015, -0.001, 0.023, -0.021, 
              -0.017, -0.075, 0.000, 0.123, 0.035, 0.051, 0.000, 0.095, -0.087, 
              -0.111, 0.000, 0.000]  
    
    zpoff3 = [0.000, 0.000, -0.196, -0.054, 0.006, 0.090, 0.043, 0.071, 0.118, # Farmer+EAZY
              0.078, 0.047, -0.034, 0.000, -0.199, -0.129, -0.084, -0.073, 
              -0.087, -0.124, 0.004, 0.154, -0.022, -0.030, -0.013, -0.057, 
              0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 
              0.000, -0.102, -0.044, 0.000, 0.000] 
    zpoff4 = [0.000, 0.000, 0.000, -0.021, 0.055, 0.124, 0.121, 0.121, 0.145, # Classic+EAZY
              0.085, 0.057, -0.036, 0.000, -0.133, -0.098, -0.046, -0.037, -0.038, 
              -0.062, 0.038, 0.214, 0.024, 0.022, 0.01, 0.022, 0.000, 0.000, 0.00, 
              0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.021, 0.025, 0.000, 0.000] 
    #print(len(zpoff1), len(zpoff2), len(zpoff3), len(zpoff4))
    
    filtinfo = {filt_name[i]: (filt_lambda[i]*1e4, filt_fwhm[i]*1e4, AlambdaDivEBV[i],
                               [zpoff1[i], zpoff2[i], zpoff3[i], zpoff4[i]], eazy_filtnum[i]) 
                for i in range(len(filt_name))}
    
    return filtinfo

#filtinfo = gather_cosmos2020_filters()

In [8]:
def parse_cosmos2020(qa=False, verbose=False, catversion='Farmer'):
    """Parse the COSMOS2020 catalog.
    
    See https://github.com/cosmic-dawn/cosmos2020-readcat/blob/main/COSMOS2020_readcat.ipynb

    """
    catfile = datadir+'/COSMOS2020_{}_R1_v2.1_p3.fits'.format(catversion.upper())
    mask = fitsio.read(catfile, columns='FLAG_COMBINED')
    print('Read {:,d} objects from {}'.format(len(mask), catfile))

    I = np.where(mask == 0)[0]
    cat = Table(fitsio.read(catfile, rows=I))
    print('Trimmed to {:,d} objects in an unmasked region.'.format(len(I)))

    filtinfo = gather_cosmos2020_filters()
    
    if catversion == 'Farmer':
        fluxprefix = '_FLUX'  
        fluxerrprefix = '_FLUXERR'
    else:
        fluxprefix = '_FLUX_APER2'
        fluxerrprefix = '_FLUXERR_APER2'
        
    # apply photometric offsets
    offindx = 1 # Farmer + eazy
    
    for filt in filtinfo.keys():
        fluxcol = f'{filt}{fluxprefix}'
        fluxerrcol = f'{filt}{fluxerrprefix}'

        if verbose: 
            print(f'Applying photometric offset to {fluxcol}')
        offset = filtinfo[filt][3][offindx]
        
        if fluxcol in cat.colnames:
            cat[fluxcol] *= 10**(-0.4 * offset)
            cat[fluxerrcol] *= 10**(-0.4 * offset)
        else:
            print(f'Missing column {fluxcol}')
        
    outflx = 'uJy'
       
    return cat, filtinfo

cosmos, cosmosfilters = parse_cosmos2020(verbose=False)
cosmos[:3]

OSError: FITSIO status = 104: could not open the named file
failed to find or open the following file: (ffopen)
/Users/ioannis/research/projects/fastspecfit-projects/laelbg-templates/COSMOS202
0_FARMER_R1_v2.1_p3.fits


In [ ]:
def parse_subaru(qa=False):
    """Parse the Subaru forced-photometry catalog.
    
    ### Subaru narrowband filters

    filts = ['IA427','IA464','IA484','IA505','IA527']  
    lamfilts = [4263.45, 4635.13, 4849.20, 5062.51, 5261.13]  
    lamstring= ['427','464','484','505','527']  
    widfilts = [195.805, 213.663, 222.029, 225.998, 245.700]  
    Avfilts = [4.202,3.894,3.694,3.490,3.304]
    
    IA427 - 181 Subaru_MB/IA427.dat lambda_c= 4.2600e+03 AB-Vega=-0.161 w95=223.1  
    IA464 - 183 Subaru_MB/IA464.dat lambda_c= 4.6333e+03 AB-Vega=-0.167 w95=237.8  
    IA484 - 184 Subaru_MB/IA484.dat lambda_c= 4.8473e+03 AB-Vega=-0.037 w95=249.5  
    IA505 - 185 Subaru_MB/IA505.dat lambda_c= 5.0608e+03 AB-Vega=-0.077 w95=259.0  
    IA527 - 186 Subaru_MB/IA527.dat lambda_c= 5.2593e+03 AB-Vega=-0.035 w95=281.7  

    ### Subaru broadband filters

    filts = ['g', 'r', 'i', 'z']  
    Av=[3.214, 2.165, 1.592, 1.211, 1.064]  

    HSC_g - 314 g_HSC.txt http://cosmos.astro.caltech.edu/page/filterset lambda_c=4798.2 AB-Vega=-0.089  
    HSC_r - 315 r_HSC.txt http://cosmos.astro.caltech.edu/page/filterset lambda_c=6218.4 AB-Vega=0.223  
    HSC_i - 316 i_HSC.txt http://cosmos.astro.caltech.edu/page/filterset lambda_c=7727.0 AB-Vega=0.468  
    HSC_z - 317 z_HSC.txt http://cosmos.astro.caltech.edu/page/filterset lambda_c=8908.2 AB-Vega=0.522  
    
    """
    catfile = datadir+'/Subaru_tractor_forced_all.fits'

    su = fitsio.read(catfile, columns=['brick_primary', 'maskbits', 'brightstarflag'])
    print('Read {:,d} objects from {}'.format(len(su), catfile))

    I = np.where((su['brick_primary']) & (su['maskbits'] == 0) & (su['brightstarflag'] == 0))[0]
    su = Table(fitsio.read(catfile, rows=I[:10]))
    print('Trimmed to {:,d} objects in an unmasked region.'.format(len(I)))
    
    # rename the narrow-band columns
    for band in ['i_a_l427', 'i_a_l464', 'i_a_l484', 'i_a_l505', 'i_a_l527']:
        newband = band.replace('_', '').replace('ial', 'ia')
        #print(newband)
        su[f'flux_{newband}'] = su[f'flux_{band}']
        su[f'flux_ivar_{newband}'] = su[f'flux_ivar_{band}']
    
    # Rename the "forced_flux_[griz] columns.
    for band in ['g', 'z']:
        su[f'flux_{band}'] = su[f'forced_flux_{band}']
        su[f'flux_ivar_{band}'] = su[f'forced_flux_ivar_{band}']
    
    # There are two r- and i-band photometric measurements; take the ivar-weighted average.
    for band in ['r', 'i']:
        f1 = su[f'forced_flux_{band}'].data
        f2 = su[f'forced_flux_{band}2'].data
        i1 = su[f'forced_flux_ivar_{band}'].data
        i2 = su[f'forced_flux_ivar_{band}2'].data
        
        I = np.where(np.logical_or(i1 > 0, i2 > 0))[0]
        fnew = np.sum(i1[I, None] * f1[I, None] + i2[I, None] * f2[I, None], axis=1) / np.sum(i1[I, None] + i2[I, None], axis=1)
        inew = 1 / (1 / np.sum(i1[I, None] + i2[I, None], axis=1))

        if qa:
            B = (f1[I] > 0) * (f2[I] > 0) * (fnew > 0)
            fig, ax = plt.subplots(1, 2, figsize=(12, 4))
            ax[0].scatter(22.5-2.5*np.log10(fnew[B]), -2.5*np.log10(f1[I][B]/fnew[B]), 
                          alpha=0.5, s=1, label=band)
            ax[0].scatter(22.5-2.5*np.log10(fnew[B]), -2.5*np.log10(f2[I][B]/fnew[B]), 
                          alpha=0.5, s=1, label=f'{band}2')
            ax[0].set_ylim(-2, 2)
            ax[0].set_xlim(15, 32)
            ax[0].set_xlabel(f'Average {band} (AB mag)')
            ax[0].set_ylabel('$\Delta m$ (AB mag)')
            ax[0].legend(markerscale=5)
        
            _ = ax[1].hist(-2.5*np.log10(f1[I][B]/fnew[B]), alpha=0.5, bins=100, range=(-1,1), label=band)
            _ = ax[1].hist(-2.5*np.log10(f2[I][B]/fnew[B]), alpha=0.5, bins=100, range=(-1,1), label=f'{band}2')
            ax[1].set_ylabel('Number')
            ax[1].set_xlabel('$\Delta m$ (AB mag)')
            ax[1].legend()
            fig.tight_layout()

        su[f'flux_{band}'] = np.zeros_like(f1)
        su[f'flux_ivar_{band}'] = np.zeros_like(f1)

        su[f'flux_{band}'][I] = fnew
        su[f'flux_ivar_{band}'][I] = inew
        
    # gather filter info
    filtinfo = {}
    for band, eazynum in zip(['g', 'r', 'i', 'z', 'ia427', 'ia464', 'ia484', 'ia505', 'ia527'],
                             [314, 315, 316, 317, 181, 183, 184, 185, 186]):
        filtinfo[band] = [eazynum]
        
    return su, filtinfo

In [ ]:
subaru, subarufilters = parse_subaru(qa=True)
#su['ra', 'dec', 'type', 'flux_g', 'flux_r', 'flux_i', 'flux_z', 'flux_ivar_g', 'flux_ivar_r', 'flux_ivar_i', 'flux_ivar_z']
subaru[:3]

### Match to the various merged-tertiary?? files

In [ ]:
def match_phot_to_desi(phot, racolumn='ra', deccolumn='dec', rad=1., 
                       label='Tractor COSMOS - Subaru/HSC'):
    """
    https://data.desi.lbl.gov/desi/survey/fiberassign/special/tertiary/0026/tertiary-config-0026.yaml

    """
    mergefiles = [datadir+'/merge-tertiary26-thru20230416.fits']
    for mergefile in mergefiles:
        
        tertiary = os.path.basename(mergefile).split('-')[1]
        
        desi = Table(fitsio.read(mergefile))
        print('Read {} DESI targets from {}'.format(len(desi), mergefile))
        
        c_desi = SkyCoord(ra=desi['TARGET_RA']*u.deg, dec=desi['TARGET_DEC']*u.deg)
        c_phot = SkyCoord(ra=phot[racolumn]*u.deg, dec=phot[deccolumn]*u.deg)
        
        #_, sep2d, _ = c_phot.match_to_catalog_sky(c_desi)
        #plt.hist(sep2d.arcsec, bins=100)
        
        indx_desi, indx_phot, d2d, _ = c_phot.search_around_sky(c_desi, rad*u.arcsec)
        indx_miss_desi = np.delete(np.arange(len(desi)), indx_desi)

        fig, ax = plt.subplots(figsize=(5, 5))
        ax.scatter(desi['TARGET_RA'], desi['TARGET_DEC'], s=1, zorder=0,
                   marker='o', color='k', alpha=0.5, label='DESI-{} (all, N={:,d})'.format(tertiary, len(desi)))
        ax.scatter(phot[racolumn], phot[deccolumn], s=1, zorder=1,
                   marker='o', color='orange', alpha=0.7, label='{} (all, N={:,d})'.format(label, len(phot)))
        ax.scatter(desi['TARGET_RA'][indx_desi], desi['TARGET_DEC'][indx_desi], s=1, zorder=2,
                   marker='s', color='blue', alpha=0.7, label='DESI-{} (N={:,d} matches)'.format(tertiary, len(indx_desi)))
        ax.set_xlabel('RA')
        ax.set_ylabel('Dec')
        ax.legend(markerscale=5)
        fig.tight_layout()
        
        #plt.scatter(desi['TARGET_RA'], desi['TARGET_DEC'], s=1)
        #plt.scatter(desi['TARGET_RA'][indx_desi], desi['TARGET_DEC'][indx_desi], s=1)
        #plt.hist(d2d.arcsec, bins=500, range=(0, 10))
        #plt.yscale('log')
        #plt.xlim(0, 2)

        return phot[indx_phot], desi[indx_desi], desi[indx_miss_desi]

In [ ]:
phot, desi, desi_miss = match_phot_to_desi(su, racolumn='ra', deccolumn='dec', label='Tractor COSMOS - Subaru/HSC')

In [ ]:
#desi['TARGETID', 'TARGET_RA', 'TARGET_DEC', 'TERTIARY_TARGET']
#set(desi['TERTIARY_TARGET'])
#desi[desi['TERTIARY_TARGET'] == 'LAE_SUBARU']
phot

In [ ]:
def write_speclite_filters(infilters, group_name='tmp'):
    """Write the eazy filters into speclite format.
    
    https://eazy-py.readthedocs.io/en/latest/eazy/filters.html
    https://speclite.readthedocs.io/en/latest/filters.html
    
    """
    import eazy
    import astropy.units as u
    from astropy.table import Table
    from speclite import filters as specfilters

    res = eazy.filters.FilterFile(os.path.join(eazy.utils.path_to_eazy_data(),
                                  'filters/FILTER.RES.latest'))
    
    for filt in infilters.keys():
        T = Table()
        
        wave = np.array(res[infilters[filt][-1]].wave)
        resp = np.array(res[infilters[filt][-1]].throughput)
        wave = np.hstack([np.min(wave)-0.1, wave, np.max(wave)+0.1])
        resp = np.hstack([0.0, resp, 0.0])
        
        meta = dict(group_name=group_name, band_name=f'{filt}')
        ff = specfilters.FilterResponse(wavelength=wave * u.Angstrom,
                                        response=resp, meta=meta)
        ff.save(datadir)
        print(ff, datadir)
        
        #specfilters.plot_filters(filt)#, wavelength_limits=(3500, 6200), cmap='jet')

    #filt = speclite.filters.load_filters('desi2-g1', 'desi2-g2', 'desi2-g3', 'desi2-g4', 'desi2-g5', 'decam2014-g')
    #speclite.filters.plot_filters(filt, wavelength_limits=(3500, 6200), cmap='jet')
    #plt.savefig('desi2-filters.png')    

In [ ]:
write_speclite_filters(subarufilters)

In [ ]:
subarufilters

In [ ]:
from speclite import filters
dir(filters)